In [ ]:
import numpy as np
import random
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rcParams["font.family"]="Malgun Gothic"
plt.rcParams["axes.unicode_minus"]=False

In [ ]:
IMG_SIZE=86
BATCH_SIZE=16
SEED=42
DATA_PATH="../../data/processed/02_kaggle_dataset/train"

datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3],
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_generator = datagen.flow_from_directory(
    directory=DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2))

model.add(Conv2D(96, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.00027600388247395235),
            loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
esc=EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history=model.fit(train_generator, validation_data=val_generator,
                epochs=20, callbacks=[esc], verbose=1)

In [ ]:
plt.figure(figsize=(12, 5))

# ACC
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("../../results/images/training_plot.png")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

y_true=val_generator.classes
steps=val_generator.samples // val_generator.batch_size + 1
y_pred_prob=model.predict(val_generator, steps=steps, verbose=0)
y_pred=(y_pred_prob > 0.5).astype(int)

report=classification_report(y_true, y_pred, target_names=["closed", "open"])
print(report)

with open("../../results/reports/classification_report.txt", "w") as f:
    f.write(report)

In [ ]:
class_names=list(val_generator.class_indices.keys())  # closed, open
indices=random.sample(range(len(y_true)), 10)  # 전체 val 데이터 중 10개 무작위 인덱스 선택

plt.figure(figsize=(15, 5))
for i, idx in enumerate(indices):  # 10개 인덱스에 대해 하나씩 처리
    batch_idx=idx // BATCH_SIZE  # 인덱스가 몇 번째 배치에 속해 있는지 계산
    offset=idx % BATCH_SIZE  # 해당 배치 내에서의 위치 계산

    val_generator.reset()  # generator 처음부터 리셋해서 배치 인덱스까지 이동
    for _ in range(batch_idx+1):  # 항상 같은 순서로 시작하기 위해 초기화
        images, _=next(val_generator)  # 원하는 배치까지 이동

    img=images[offset].squeeze()  # 해당 배치에서 원하는 이미지 하나 꺼내기(squeeze: 86, 86, 1 -> 86, 86)
    true_label=class_names[int(y_true[idx])]  # real
    pred_label=class_names[int(y_pred[idx])]  # pred

    plt.subplot(2, 5, i+1)
    plt.imshow(img, cmap="gray")
    plt.title(f"실제: {true_label} / 예측: {pred_label}")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬
cm=confusion_matrix(y_true, y_pred)

plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("예측 라벨")
plt.ylabel("실제 라벨")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("../../results/images/confusion_matrix.png")
plt.show()

In [ ]:
model_path="../../src/models/02_Basic_Model/final_model_from_tuner.keras"
model.save(model_path)
print(f"최종 모델 저장 완료: {model_path}")

In [ ]:
print("최종 Validation Accuracy:", history.history["val_accuracy"][-1])
print("모델 저장 완료 및 리포트 저장 완료")